In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import json

model = ChatOllama(
    model="llama3.1",
)

In [ ]:
user_prompt = "Teach me about making games in C"

# Generate Initial Course Data


In [ ]:
welcome_prompt = """Part 1: Context Setting
You are an expert curriculum designer with extensive knowledge across various subjects. Your task is to create a comprehensive course outline based on the user's question or topic of interest. The course should be suitable for self-study and cover all necessary aspects of the subject.
Part 2: Input Processing
User Input: [Insert user's question or topic here]
Based on the user's input, identify the main subject area, the depth of knowledge required, and any specific aspects the user might be interested in. Consider the potential background of someone asking this question and tailor the course accordingly.
Part 3: Output Instructions
Generate a full course outline in markdown format, including:

Course Title
Course Description (2-3 sentences)
Learning Objectives (3-5 bullet points)
Prerequisites (if any)
Module Breakdown (5-8 modules)
For each module, include:

Module title
Brief description
Key topics covered (3-5 bullet points)
Suggested resources (1-2 books, articles, or online materials)


Assessment Methods (quizzes, projects, final exam, etc.)
Additional Resources (optional further reading or study materials)

Ensure the course structure is logical, progressive, and comprehensive. Use clear, concise language suitable for the target audience. Format the output in proper markdown for easy readability and potential integration into learning management systems."""

In [ ]:
welcome_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            welcome_prompt,
        ),
        ("human", "{input}"),
    ]
)

welcome_chain = welcome_prompt_template | model

In [ ]:
welcome_data_str = ""
for chunk in welcome_chain.stream(
    {
        "input": user_prompt,
    }
):
    welcome_data_str += chunk.content
    print(chunk.content, end="", flush=True)

In [ ]:
course_welcome_data = json.loads(welcome_data_str)

In [ ]:
print(course_welcome_data["course_name"])
print(course_welcome_data["description"])

# Generate an outline prompt


In [ ]:
outline_prompt = """Given the following course title and description, generate a comma-separated list of 5-10 logical sections 
or modules that would make up a comprehensive curriculum for that course. 
Ensure the sections flow in a sensible order from beginner to more advanced topics. 
The sections or modules should appear as natural language. 
Format the response as a single line of text with no line breaks. Do not say anything else."""

In [ ]:
outline_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            outline_prompt,
        ),
        ("human", "{input}"),
    ]
)

outline_chain = outline_prompt_template | model

In [ ]:
sections = ""
for chunk in outline_chain.stream(
    {
        "input": f"{course_welcome_data["course_name"]} {course_welcome_data["description"]}",
    }
):
    sections += chunk.content
    print(chunk.content, end="", flush=True)

In [ ]:
sections = sections.split(",")

# Generate modules prompt


In [ ]:
section_prompt = """You are an expert curriculum designer and educational content creator. 
Your task is to create a detailed content structure for a course module or chapter. Please follow these guidelines: 
Structure Requirements: 
Begin with an introduction to the topic. 
Include 5-10 main sections or comprehensive lessons. 
Ensure a logical flow from basic to more advanced concepts. 
Incorporate diverse learning activities, exercises, solutions, and assessments. 
End with a summary and connection to broader course themes. 
For each main section or lesson: 
Provide a clear, engaging title. 
Include at least 5-10 sentences about the section or lesson. 
The user should fully understand the lesson in detail after reading the section.
Please generate a comprehensive content structure based on these guidelines given the chapter title and the other course context provided below. 
Your response should be in a clear, hierarchical format that an educator or instructional designer could easily follow and adapt. 
Ensure your output is in Markdown format and do not say anything else."""

In [ ]:
section_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            section_prompt,
        ),
        (
            "human",
            "{input}",
        ),
    ]
)

section_chain = section_prompt_template | model

In [ ]:
documents = []
for i, section in enumerate(sections):
    document = {"title": section, "content": ""}
    for chunk in section_chain.stream(
        {
            "input": f"""Course Title: { course_welcome_data["course_name"]}\nCourse Description: { course_welcome_data["description"]}\n Previously covered chapters: {sections[i:]}\nChapter Title: {section}\nChapter Number:{i+1}"""
        }
    ):
        document["content"] += chunk.content
        print(chunk.content, end="", flush=True)
    documents.append(document)

In [ ]:
from IPython.display import display, Markdown

for document in documents:
    display(Markdown(document["content"]))